In [1]:
import os, getpass

# Prompt for key if missing
if not os.environ.get("OPENROUTER_API_KEY"):
    os.environ["OPENROUTER_API_KEY"] = getpass.getpass("OPENROUTER_API_KEY: ")

### Basic Agent Example

In [2]:
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.llms.openrouter import OpenRouter

llm = OpenRouter(
    api_key=os.environ["OPENROUTER_API_KEY"],
    model="anthropic/claude-3.5-sonnet",
    is_chat_model=True,
    is_function_calling_model=True
)

# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b


# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=llm,
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)

### Adding Chat History

In [3]:
from llama_index.core.workflow import Context

# create context
ctx = Context(agent)

# run agent with context
response = await agent.run("My name is Logan", ctx=ctx)
print(str(response))

response = await agent.run("What is my name?", ctx=ctx)
print(str(response))

Hello Logan! I am an assistant that can help you multiply numbers together. Would you like me to multiply any numbers for you? Just let me know what numbers you'd like to multiply and I'll help you calculate the result.
Your name is Logan.


### Maintaining state over longer periods

In [4]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

ctx_dict = ctx.to_dict(serializer=JsonSerializer())

restored_ctx = Context.from_dict(
    agent, ctx_dict, serializer=JsonSerializer()
)

response3 = await agent.run(user_msg="What's my name?", ctx=restored_ctx)
print(str(response3))

Your name is Logan.


### Tools and state

In [5]:
async def set_name(ctx: Context, name: str) -> str:
    async with ctx.store.edit_state() as ctx_state:
        ctx_state["state"]["name"] = name

    return f"Name set to {name}"

workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

ctx = Context(workflow)

# check if it knows a name before setting it
response = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(str(response))

response2 = await workflow.run(user_msg="My name is Laurie", ctx=ctx)
print(str(response2))

state = await ctx.store.get("state")
print("Name as stored in state: ", state["name"])


Based on the current state, your name is "unset". Would you like to set your name to something else? If so, please let me know what name you'd like to use, and I can help you set it using the set_name function.
Your name has been set to Laurie!
Name as stored in state:  Laurie
